In [1]:
from IPython.display import display, Math, Latex
import pandas as pd
import numpy as np
#import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import itertools
import copy
from datetime import datetime

In [2]:
MinStocks = 10
MaxStocks = 22
MaxWeight = 0.2
MinWeight = 0.05
Start_date = "2023-01-01"
End_date = "2023-06-30"
TradingFee = 4.95
InvestmentAmount = 750000
ticker_file = "example_tickers2.csv"

In [3]:
tickers = pd.read_csv(ticker_file, header=None)##Header is none to read in the first element.
tickers.columns=["Tickers"]
exchangeCADUSD = yf.download('CADUSD=x', start=Start_date, end=End_date).tz_localize(None)
tickers

[*********************100%%**********************]  1 of 1 completed


,Tickers
0,CNR.TO
1,CP.TO
2,TRI.TO
3,WCN.TO
4,WSP.TO
5,AC.TO
6,TIH.TO
7,TFII.TO
8,SNC.TO
9,CAE.TO


In [4]:
def USDtoCAD(closing):
    #Merges the closing data with the exchange data, then removes all na values (This makes sure each closing price is matched with its dates exchange rate)
    exchanged = pd.concat([closing.to_frame(),exchangeCADUSD["Close"]],axis=1).dropna()

    #Since we are going USD to CAD, we divide by the exchange rate of that day
    converted = exchanged.iloc[:,0]/exchanged.iloc[:,1]
    return converted


In [5]:
## Consumes a series or listof possible stock symboles
def validTickers (tickers):
    ##Creates a new dataframe to hold the valid tickers closing prices
    df = pd.DataFrame()

    #Itterating through the tickers

    for i in tickers["Tickers"]:
        #Creating the ticker objects
        ticker = yf.Ticker(i)

        #Try ensures that even if an error occurs, the code will still run (this gets past the delisted stocks)
        try:
            #Running .fast_info['currency'] on a stock will cause an error and stop any non-listed stocks from being ran
            ticker.fast_info['currency']

            #Download the Daily stock history for the given time periods
            history = ticker.history(start=Start_date, end=End_date).tz_localize(None)

            #Creates a seperate dataframe with the values collected by month
            histByMonth = history['Volume'].to_frame().groupby([(history.index.month)])

            #Drops any months with less than 18 trading days
            histByMonth = histByMonth.filter(lambda x: len(x) > 17)

            #Checks for an average monthly volume greater than 150,000
            if histByMonth.sum().mean() > 150000:
                #Checking if the stock is listed in CAD or USD
                if ticker.fast_info['currency'] == 'CAD':

                    #Adding the stock to the valid tickers dataframe
                    df[i] = history['Close']

                elif ticker.fast_info['currency'] == 'USD':

                    #Adding the stocks closing data to the valid tickers dataframe after converting the prices to CAD
                    df[i] = USDtoCAD(history['Close'])
        except:
            #skips and tickers which cause an error
            pass
    return df

In [6]:
stocks_close = validTickers(tickers)
stocks_close

,CNR.TO,CP.TO,TRI.TO,WCN.TO,WSP.TO,AC.TO,TIH.TO,TFII.TO,CAE.TO,BHC.TO,...,RY.TO,BNS.TO,NA.TO,TD.TO,BMO.TO,CM.TO,MFC.TO,IFC.TO,SLF.TO,FFH.TO
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,160.412766,101.791740,149.512558,179.046158,156.854950,19.120001,96.576591,135.544189,26.610001,8.59,...,122.739281,63.176792,88.789734,84.689217,118.579041,52.857281,23.325953,192.465256,61.336895,807.609985
2023-01-04,160.580200,102.786194,150.764267,177.673706,157.162964,19.760000,97.456268,136.456131,26.879999,9.27,...,123.832169,64.359116,90.106865,85.790459,120.255875,53.410660,23.594175,193.438522,61.926678,806.900024
2023-01-05,158.088531,100.648125,147.796188,173.695557,155.553284,20.120001,99.581329,134.037476,26.559999,9.72,...,123.467873,64.465714,89.729164,84.384216,120.036743,53.286625,23.412167,193.182907,61.220875,801.000000
2023-01-06,161.949142,103.641411,148.772919,177.514587,158.693176,20.200001,100.955208,138.339508,26.799999,10.04,...,125.040100,65.115021,91.007545,84.345154,121.827911,53.964043,23.642073,196.427032,61.655956,807.830017
2023-01-09,162.313538,103.452469,147.521225,176.191849,162.876358,20.500000,100.194145,139.925522,27.030001,10.09,...,125.970001,65.211929,91.278725,84.081490,121.685005,54.212109,23.613337,194.510056,61.037174,806.849976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-23,154.498108,105.178032,174.870789,179.306305,167.906784,22.879999,103.899849,138.731155,28.000000,9.46,...,119.588829,62.267979,93.581787,77.080322,112.737320,53.517017,23.658611,194.636230,65.632553,965.070007
2023-06-26,155.413223,104.719742,176.597961,180.863495,168.932480,22.860001,102.594872,139.775085,28.040001,9.56,...,120.859810,63.015823,95.193901,78.345230,114.364563,54.572830,23.747257,195.869217,66.620995,978.979980
2023-06-27,156.775970,105.486877,181.144196,182.580399,170.436172,23.760000,105.324364,150.820679,28.180000,9.73,...,122.433861,63.547188,95.965340,79.106155,115.718971,54.917168,24.121540,197.997147,66.927406,992.119995


In [7]:
def USDtoCAD(closing):
    #Merges the closing data with the exchange data, then removes all na values (This makes sure each closing price is matched with its dates exchange rate)
    exchanged = pd.concat([closing.to_frame(),exchangeCADUSD["Close"]],axis=1).dropna()

    #Since we are going USD to CAD, we divide by the exchange rate of that day
    converted = exchanged.iloc[:,0]/exchanged.iloc[:,1]
    return converted


In [8]:
corr = stocks_close.corr()##Gets correlation between each stock pair
valid_tickers = list(stocks_close.columns.values)
corr

,CNR.TO,CP.TO,TRI.TO,WCN.TO,WSP.TO,AC.TO,TIH.TO,TFII.TO,CAE.TO,BHC.TO,...,RY.TO,BNS.TO,NA.TO,TD.TO,BMO.TO,CM.TO,MFC.TO,IFC.TO,SLF.TO,FFH.TO
CNR.TO,1.000000,0.504252,-0.089355,0.136582,0.127080,-0.247459,-0.242738,-0.081589,0.139408,-0.277023,...,0.339757,0.059964,0.103876,0.198588,0.186689,-0.155911,-0.043382,0.205645,-0.325625,-0.504396
CP.TO,0.504252,1.000000,0.137667,0.484609,0.457270,0.028974,0.115308,-0.058164,0.157621,-0.270713,...,0.131289,0.048364,0.503394,0.017100,-0.045602,-0.088891,0.269124,0.481852,0.153176,0.185026
TRI.TO,-0.089355,0.137667,1.000000,0.630952,0.414747,-0.263950,0.303045,0.097527,0.278700,-0.068874,...,-0.337190,-0.413278,0.045502,-0.629193,-0.592662,-0.253594,-0.051376,0.284545,0.083846,0.559675
WCN.TO,0.136582,0.484609,0.630952,1.000000,0.592553,-0.422465,0.300722,0.110437,0.427157,-0.238495,...,-0.145483,-0.240142,0.355207,-0.493267,-0.556218,-0.250827,0.305700,0.438471,0.022663,0.503150
WSP.TO,0.127080,0.457270,0.414747,0.592553,1.000000,-0.130930,0.473951,0.119957,0.451599,-0.123118,...,0.108749,0.020857,0.583913,-0.253286,-0.239125,-0.004266,0.544790,0.375470,0.252934,0.487810
AC.TO,-0.247459,0.028974,-0.263950,-0.422465,-0.130930,1.000000,-0.130980,-0.278579,-0.319438,-0.024936,...,-0.098078,0.231691,0.284377,0.296743,0.287217,0.261563,0.109037,0.203224,0.736982,0.232486
TIH.TO,-0.242738,0.115308,0.303045,0.300722,0.473951,-0.130980,1.000000,0.631487,0.556677,0.624434,...,0.290563,0.212302,0.487939,0.074563,0.080435,0.503653,0.589142,0.042800,0.285158,0.457264
TFII.TO,-0.081589,-0.058164,0.097527,0.110437,0.119957,-0.278579,0.631487,1.000000,0.621848,0.583755,...,0.585428,0.464930,0.218548,0.339542,0.388372,0.676179,0.298149,-0.394184,-0.055139,-0.059176
CAE.TO,0.139408,0.157621,0.278700,0.427157,0.451599,-0.319438,0.556677,0.621848,1.000000,0.339969,...,0.591399,0.531813,0.510356,0.253740,0.265104,0.516404,0.522368,0.118844,0.061897,0.032615
BHC.TO,-0.277023,-0.270713,-0.068874,-0.238495,-0.123118,-0.024936,0.624434,0.583755,0.339969,1.000000,...,0.316857,0.328696,0.099608,0.349223,0.378250,0.594851,0.291255,-0.207612,0.110074,0.002301


In [9]:
#Takes a list of tickers and returns the average correlation among them
def avg_corr(tickers):
    #amount of tickers
    amount = len(tickers)
    #In the case where you only give one ticker, we define
    #The correlation to 1, even though there is nothing to
    #correlated to. This is just here for failsafe as
    #the code would run into a divide by 0 error later for
    #the case where amount = 1.
    if amount == 0:
        return 1;
    sum = 0
    for i in tickers:
        for j in tickers:
            if not i==j:
              sum+=corr[i][j]
    #The average is the sum of each correlation pair divided by the
    #total amount of pairs compared which can be represented as n*(n-1)
    #Yes you can technically reduce the amount of comparisons by 50%
    #but this won't be a problem unless there are like 10000+ stocks.
    average = sum/(amount*(amount-1))
    return average


max_corr = -1
selected = []

#Given an amount to brute force, a list of stocks and a ticker,
#it returns a list of tickers of length n+1, where it includes
#n+1 stocks with the highest average correlation between them,
#given that the ticker given is 1 of the stocks in the list.
#n=amount of tickers to brute force
#selections= list of all tickers you can choose from
#stock= ticker that has to be included in the returned list
def brute_force(n, selections, stock):
    #Get all combos of the selections.
    combos = (list(itertools.combinations(valid_tickers,n)))
    max_corr = -1
    selected = []
    #We check each list in combo to see which has the highest
    #average correlation once you add the given ticker
    for i in combos:
        #We don't want to include the given ticker twice
        if not stock in i:
            temp = list(i)
            temp.append(stock)
            corr = avg_corr(temp)
            #If this list's average correlation is larger
            #than the current max, update the max correlation.
            if corr>max_corr:
                max_corr = corr
                selected = temp
    return selected

#Given how many stocks to brute force for highest average
#correlation and given how many total stocks you want
#in your portfolio, and the list of all options of tickers
#you can choose from, give a rough estimate for
#a portfolio with total amount of stocks with the highest
#average correlation between them.
#We are estimating the best portfolio because
def estimate_best_corr(bf, total, selections):
    #We subtract 1 from the amount we brute force, because
    #we will run the brute_force function on each ticker
    #which creates a list of stocks of length n+1, so
    #substituting bf-1 into n will give us
    #a list of length (bf-1)+1 = bf which is ultimately
    #how many stocks we want
    bf = bf-1
    max_corr = -1
    estimated_selected = []
    #We will run the brute force function on each stock.
    for i in selections:
        #Brute_force on said stock
        selected = brute_force(bf, selections, i)
        best_selected = selected
        #For the remaining stocks we need to add to the portfolio
        #we just repeatedly add the stock that creates the highest
        #possible average correlation when added to the list
        #of stocks we got from brute_force
        for j in range(total-bf-1):
            temp_max_corr = -1
            #Try adding each stock into the portfolio and see which
            #one gives us the highest average correlation.
            for k in selections:
                #We don't want to add the stock if it
                #is already in our portfolio
                if not k in selected:
                    #Create a temp portfolio with the stock added
                    temp_selected = copy.copy(selected)
                    temp_selected.append(k)
                    corr = avg_corr(temp_selected)
                    #If the average correlation is the highest
                    #we've seen, we update the max correlation
                    #and our best new portfolio
                    if corr>temp_max_corr:
                        temp_max_corr = corr
                        best_selected = copy.copy(temp_selected)
            #Update portfolio
            selected = copy.copy(best_selected)
        corr = avg_corr(selected)
        if corr>max_corr:
            max_corr=corr
            estimated_selected = selected
    return estimated_selected

portfolio = estimate_best_corr(3, 10, valid_tickers)
print(portfolio, avg_corr(portfolio))

['ERF.TO', 'CVE.TO', 'SU.TO', 'PPL.TO', 'OGI.TO', 'CTS.TO', 'ACB.TO', 'WEED.TO', 'BMO.TO', 'TD.TO'] 0.8377151360818287


In [10]:
#Returns average weekly return of a stock
def weekly_return(ticker):
    close = stocks_close[ticker]
    returns=close.resample('W').ffill().pct_change()
    return returns.mean()

#Orders the portfolio in terms of which one we want to give more weight,
#and the larger the magnitude of the weekly return is, the higher
#it is prioritized.
def rankings(portfolio):
    #First puts all the tickers and their weekly returns
    #into a dictionary
    rank = {}
    for i in portfolio:
        rank[i] = weekly_return(i)
    #Sort the dictionary
    keys = list(rank.keys())
    values = list(rank.values())
    sorted_index = np.argsort(values)
    ordered = {keys[i]: values[i] for i in sorted_index}

    #Since it is in ascending order at this moment
    #in the case where the returns are negative,
    #we leave the array as it is, as the stock
    #with the lowest return (or highest magnitude)
    #is already ranked first. But if the returns are positive
    #the dict would be fliped, so if the first returns is positive,
    #we flip the dictionary
    first_val = next(iter(ordered.values()))
    if first_val>0:
      ordered = dict(reversed(list(ordered.items())))
    df = pd.DataFrame(ordered.items())
    return df


ranks = rankings(portfolio)
ranks

,0,1
0,WEED.TO,-0.061352
1,OGI.TO,-0.029424
2,ACB.TO,-0.016241
3,CTS.TO,-0.012987
4,ERF.TO,-0.003546
5,CVE.TO,-0.003360
6,PPL.TO,-0.002776
7,BMO.TO,-0.001795
8,TD.TO,-0.001447
9,SU.TO,-0.001437


In [11]:
ranked = ranks[0].squeeze()
tickerObjects = []
##Converting strings to Tickers
for i in ranked:
  ticker = yf.Ticker(i)
  tickerObjects = tickerObjects+ [ticker]


In [12]:
#Putting the highest possible weight into as few stocks as possible
#returns a list of 10 numbers that represent how much we put into each stock
def BestWeights(numStocks,MaxWeight,MinWeight,InvestmentAmount):
    RemainingAmount = InvestmentAmount - TradingFee*numStocks #subtracting the trading fee each time
    RemainingStocks = numStocks
    MaxAmount = InvestmentAmount*MaxWeight
    MinAmount = InvestmentAmount*MinWeight
    Amounts = []
    #allocates the maximum weight (MaxAmount) to as few stocks as possible
    #until the remaining amount allows it and there are more than one stock remaining
    while RemainingAmount > MinAmount*(RemainingStocks+1) and RemainingStocks > 1:
        RemainingStocks = RemainingStocks - 1
        RemainingAmount = RemainingAmount - MaxAmount
        Amounts = Amounts + [MaxAmount]
    #when the above loop ends (due to insufficient remaining amount or only one stock remaining)
    #we assign the minimum weight (MinAmount) to the remaining stocks
    while RemainingStocks > 1:
        RemainingStocks = RemainingStocks - 1
        RemainingAmount = RemainingAmount - MinAmount
        Amounts = Amounts + [MinAmount]

    Amounts = Amounts + [RemainingAmount]
    sortedAmounts = sorted(Amounts, reverse=True) #sorting the list
    return sortedAmounts


WeightedAmounts = BestWeights(10,MaxWeight,MinWeight,InvestmentAmount)
WeightedAmounts

[150000.0,
 150000.0,
 150000.0,
 74950.5,
 37500.0,
 37500.0,
 37500.0,
 37500.0,
 37500.0,
 37500.0]

In [13]:
Portfolio_Final = pd.DataFrame()
Portfolio_Final["Ticker"] = pd.concat([Portfolio_Final,ranks.iloc[:,0]],)

counter = 0
Portfolio_Final["Currency"] = Portfolio_Final["Ticker"]
for i in tickerObjects:
  Portfolio_Final["Currency"][counter] = i.fast_info['currency']
  counter = counter + 1

counter = 0
Portfolio_Final["Shares"] = Portfolio_Final["Ticker"]
exchange = yf.Ticker("CADUSD=x")
for i in tickerObjects:
  if i.fast_info['currency'] == 'USD':
    Portfolio_Final["Shares"][counter] = (WeightedAmounts[counter]*exchange.fast_info["lastPrice"])/i.fast_info['lastPrice']
  else:
    Portfolio_Final["Shares"][counter] = WeightedAmounts[counter]/i.fast_info['lastPrice']
  counter = counter + 1

counter = 0
Portfolio_Final["Value"] = Portfolio_Final["Ticker"]
for i in tickerObjects:
  Portfolio_Final["Value"][counter] = WeightedAmounts[counter]
  counter = counter + 1

counter = 0
Portfolio_Final["Weight"] = Portfolio_Final["Ticker"]
for i in tickerObjects:
  Portfolio_Final["Weight"][counter] = round((WeightedAmounts[counter]/(InvestmentAmount-(10*TradingFee))*100),2)
  counter = counter + 1


Portfolio_Final

,Ticker,Currency,Shares,Value,Weight
0,WEED.TO,CAD,185185.18464,150000.0,20.0
1,OGI.TO,CAD,84745.763625,150000.0,20.0
2,ACB.TO,CAD,229007.643591,150000.0,20.0
3,CTS.TO,CAD,18103.986091,74950.5,9.99
4,ERF.TO,CAD,1691.47496,37500.0,5.0
5,CVE.TO,CAD,1500.30001,37500.0,5.0
6,PPL.TO,CAD,839.677567,37500.0,5.0
7,BMO.TO,CAD,335.450395,37500.0,5.0
8,TD.TO,CAD,440.709856,37500.0,5.0
9,SU.TO,CAD,815.217391,37500.0,5.0
